일단 1125_test_hsh를 진행한 결과 학습이 제대로 이루어지지 않음
고로 계층의 깊이가 너무 깊었기 때문에 생기는 문제점, 흑백 사진으로 인한 동공 이미지 사진의 정보를 잃었을 가능성 때문이라고 판단해
resnet50 -> resnet18 로 진행, 그리고 grayscale -> rgb로 그대로 사용할 것.

In [1]:
import os
import re
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, Model, Input

# Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

# 이미지 로드 및 전처리
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='rgb', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 파일 이름에서 session과 point 추출
def extract_session_and_point(filename):
    session_match = re.search(r'img_(\d+)', filename)
    point_match = re.search(r'\((\d+)\)', filename)
    session = int(session_match.group(1)) if session_match else None
    point = int(point_match.group(1)) if point_match else None
    return session, point

# ResNet18 모델 생성
def create_resnet18_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    filter_sizes = [64, 128, 256, 512]
    num_blocks = [2, 2, 2, 2]
    for filters, blocks in zip(filter_sizes, num_blocks):
        for i in range(blocks):
            x = resnet_block(x, filters, downsample=(i == 0 and filters != 64))
    x = layers.GlobalAveragePooling2D()(x)
    return Model(inputs=input_layer, outputs=x)

# ResNet 블록 생성
def resnet_block(x, filters, downsample=False):
    shortcut = x
    strides = (2, 2) if downsample else (1, 1)
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if downsample or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
    x = layers.add([x, shortcut])
    x = layers.Activation(mish)(x)
    return x

# MLP 모델 생성
def create_mlp_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Dense(128, activation=mish)(input_layer)
    x = layers.Dense(64, activation=mish)(x)
    x = layers.Dense(3, activation=mish)(x)
    x = layers.Flatten()(x)
    return Model(inputs=input_layer, outputs=x)

# 유효한 라벨 정의
valid_labels = [1, 5, 9, 12, 16, 19, 23, 27, 30, 34, 37, 41, 45]

# 데이터 로드 및 전처리
def prepare_data(folder_path, csv_path, test_subject, valid_labels):
    subject_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    train_subjects = [s for s in subject_folders if s != test_subject]
    
    # 학습 데이터 로드
    train_images, train_csv_data = [], pd.DataFrame()
    for train_subject in train_subjects:
        train_folder = os.path.join(folder_path, train_subject)
        train_csv = os.path.join(csv_path, f"{train_subject}.csv")
        train_images.extend(glob.glob(os.path.join(train_folder, '*.jpg')))
        if os.path.exists(train_csv):
            train_csv_data = pd.concat([train_csv_data, pd.read_csv(train_csv)])
    train_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 테스트 데이터 로드
    test_folder = os.path.join(folder_path, test_subject)
    test_csv = os.path.join(csv_path, f"{test_subject}.csv")
    test_images = glob.glob(os.path.join(test_folder, '*.jpg'))
    test_csv_data = pd.read_csv(test_csv) if os.path.exists(test_csv) else pd.DataFrame(columns=['Session', 'Point'])
    test_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 데이터 처리 함수
    def process_images(image_paths, csv_data, valid_labels):
        image_data = []
        for img in image_paths:
            session, point = extract_session_and_point(os.path.basename(img))
            if point in valid_labels:
                subject_name = os.path.basename(os.path.dirname(img))
                unique_filename = f"{subject_name}_{os.path.basename(img)}"
                image_data.append({
                    'Filename': os.path.abspath(img),
                    'UniqueFilename': unique_filename,
                    'Session': session,
                    'Point': point
                })
        df = pd.DataFrame(image_data)
        merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
        merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
        merged = merged[merged['Point'].isin(valid_labels)]
        return merged
    
    train_df = process_images(train_images, train_csv_data, valid_labels)
    test_df = process_images(test_images, test_csv_data, valid_labels)
    return train_df, test_df

# 모델 훈련 및 평가
def train_and_evaluate(folder_path, csv_path, valid_labels, test_subjects):
    accuracies = []
    for test_subject in test_subjects:
        print(f"Testing on subject: {test_subject}")
        train_df, test_df = prepare_data(folder_path, csv_path, test_subject, valid_labels)
        
        train_images_array = np.array([load_and_preprocess_image(path) for path in train_df['Filename']])
        train_features = train_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
        train_labels = train_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
        train_labels = to_categorical(train_labels, num_classes=len(valid_labels))
        
        test_images_array = np.array([load_and_preprocess_image(path) for path in test_df['Filename']])
        test_features = test_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
        test_labels = test_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
        test_labels = to_categorical(test_labels, num_classes=len(valid_labels))
        
        right_eye_model = create_resnet18_model((128, 128, 3))
        left_eye_model = create_resnet18_model((128, 128, 3))
        mlp_model = create_mlp_model(train_features.shape[1:])
        combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output, mlp_model.output])
        x = layers.Dense(256, activation=mish)(combined_input)
        x = layers.Dropout(0.5)(x)
        output_layer = layers.Dense(len(valid_labels), activation='softmax')(x)
        combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input, mlp_model.input], outputs=output_layer)

        combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        history = combined_model.fit(
            [train_images_array, train_images_array, train_features], train_labels,
            validation_data=([test_images_array, test_images_array, test_features], test_labels),
            epochs=50,
            batch_size=1,
            verbose=1
        )
        
        predictions = combined_model.predict([test_images_array, test_images_array, test_features])
        predicted_classes = np.argmax(predictions, axis=1)
        actual_classes = np.argmax(test_labels, axis=1)

        loss, accuracy = combined_model.evaluate([test_images_array, test_images_array, test_features], test_labels)
        accuracies.append(accuracy)
        print(f"Testing accuracy for subject {test_subject}: {accuracy:.4f}")
        
        # 결과 저장
        results = []
        for idx, (image_path, feature, pred_class, actual_class) in enumerate(zip(
            test_df['Filename'], test_features, predicted_classes, actual_classes)):
            results.append({
                'Subject': test_subject,
                'Test Accuracy': accuracy,
                'Test Loss': loss,
                'Image File': image_path,
                'Feature': feature.tolist(),
                'Predicted Class': pred_class,
                'Actual Class': actual_class
            })
        
        results_df = pd.DataFrame(results)
        results_df.to_csv(os.path.join(csv_path, f"results_{test_subject}.csv"), index=False, encoding='utf-8')
        print(f"Results for subject {test_subject} saved.")
        
        # 모델 저장
        model_save_path = os.path.join(csv_path, f"model_{test_subject}.h5")
        combined_model.save(model_save_path)
        print(f"Model saved at: {model_save_path}")
    
    print(f"Average Accuracy: {np.mean(accuracies):.4f}")

# 데이터 경로
folder_path = r"C:\Users\admin\Desktop\sihoon\webcam\img"
csv_path = r"C:\Users\admin\Desktop\sihoon\webcam\results"
test_subjects = ['hsh', 'hyh']  # 테스트로 사용할 대상

# LOSO 수행
train_and_evaluate(folder_path, csv_path, valid_labels, test_subjects)


Testing on subject: hsh
Epoch 1/50


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_65', 'keras_tensor_130']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


2600/2600 ━━━━━━━━━━━━━━━━━━━━ 425s 157ms/step - accuracy: 0.1860 - loss: 2.6892 - val_accuracy: 0.1000 - val_loss: 2.9397
Epoch 2/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 409s 157ms/step - accuracy: 0.6300 - loss: 0.9870 - val_accuracy: 0.2077 - val_loss: 6.1664
Epoch 3/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 417s 160ms/step - accuracy: 0.8040 - loss: 0.5774 - val_accuracy: 0.2615 - val_loss: 6.6334
Epoch 4/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 400s 154ms/step - accuracy: 0.8614 - loss: 0.4520 - val_accuracy: 0.0538 - val_loss: 9.5488
Epoch 5/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 399s 153ms/step - accuracy: 0.8885 - loss: 0.3158 - val_accuracy: 0.1354 - val_loss: 16.8776
Epoch 6/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 399s 153ms/step - accuracy: 0.9146 - loss: 0.2688 - val_accuracy: 0.1969 - val_loss: 8.0665
Epoch 7/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 409s 157ms/step - accuracy: 0.9270 - loss: 0.2480 - val_accuracy: 0.2631 - val_loss: 16.1317
Epoch 8/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 413s 159ms/step - accuracy: 0

Testing accuracy for subject hsh: 0.2308
Results for subject hsh saved.
Model saved at: C:\Users\admin\Desktop\sihoon\webcam\results\model_hsh.h5
Testing on subject: hyh
Epoch 1/50


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_139', 'keras_tensor_204', 'keras_tensor_269']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


2600/2600 ━━━━━━━━━━━━━━━━━━━━ 431s 160ms/step - accuracy: 0.2281 - loss: 2.6586 - val_accuracy: 0.5492 - val_loss: 1.3062
Epoch 2/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 415s 159ms/step - accuracy: 0.6249 - loss: 1.0113 - val_accuracy: 0.3892 - val_loss: 2.2551
Epoch 3/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 414s 159ms/step - accuracy: 0.7936 - loss: 0.5272 - val_accuracy: 0.5815 - val_loss: 2.9172
Epoch 4/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 414s 159ms/step - accuracy: 0.8874 - loss: 0.3408 - val_accuracy: 0.4862 - val_loss: 2.8219
Epoch 5/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 415s 159ms/step - accuracy: 0.9053 - loss: 0.2949 - val_accuracy: 0.5292 - val_loss: 2.2479
Epoch 6/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 414s 159ms/step - accuracy: 0.9237 - loss: 0.2684 - val_accuracy: 0.5923 - val_loss: 1.7755
Epoch 7/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 415s 160ms/step - accuracy: 0.9350 - loss: 0.2121 - val_accuracy: 0.4985 - val_loss: 2.6425
Epoch 8/50
2600/2600 ━━━━━━━━━━━━━━━━━━━━ 416s 160ms/step - accuracy: 0.9

Testing accuracy for subject hyh: 0.3538
Results for subject hyh saved.
Model saved at: C:\Users\admin\Desktop\sihoon\webcam\results\model_hyh.h5
Average Accuracy: 0.2923
